Load the data

In [149]:
import pandas as pd

df = pd.read_csv('../data/1_raw/raw_data.csv')
display(df.head())
display(df.columns)
display(df.shape)


,Property ID,Locality name,Postal code,Price,Type of property,Subtype of property,Number of rooms,Living area,Equipped kitchen,Furnished,Open fire,Terrace,Garden,Number of facades,Swimming pool,State of building,Garden Surface,Terrace Surface,Price per m2
0,VWD15538,Seneffe,7180,410000,House,Villa,4.0,175.0,1,0,0,1,1,4.0,0,Normal,NaN,NaN,2342.857143
1,RBU63284,Hennuyères,7090,750000,House,House,4.0,550.0,0,0,0,1,1,4.0,0,Normal,1085.0,30.0,1363.636364
2,VBD47252,La Louvière,7100,175000,Appartment,Flat,2.0,76.0,1,0,0,1,0,2.0,0,Unknown,NaN,NaN,2302.631579
3,RBU62400,Solre-sur-Sambre,6560,80000,House,House,3.0,235.0,0,0,0,1,0,3.0,0,To be renovated,NaN,30.0,340.425532
4,RBU62399,Mons,7000,195000,House,House,2.0,103.0,0,0,0,1,1,NaN,0,Unknown,80.0,10.0,1893.203883


Index(['Property ID', 'Locality name', 'Postal code', 'Price',
       'Type of property', 'Subtype of property', 'Number of rooms',
       'Living area', 'Equipped kitchen', 'Furnished', 'Open fire', 'Terrace',
       'Garden', 'Number of facades', 'Swimming pool', 'State of building',
       'Garden Surface', 'Terrace Surface', 'Price per m2'],
      dtype='object')

(6850, 19)

Clean data
- handling missing values
- removing duplicates
- handling outliers
- fixing data types

I want to rename the column names with snake case

In [150]:
import re

df_clean = df.copy()

def to_snake_case(name):
    name = name.strip().lower()
    name = re.sub(r'[^a-z0-9]+', '_', name)     # replace non-alphanumerics with _
    name = re.sub(r'_+', '_', name)             # remove repeated underscores
    return name

df_clean = df_clean.rename(columns=lambda x: to_snake_case(x))
display(df_clean.columns)

Index(['property_id', 'locality_name', 'postal_code', 'price',
       'type_of_property', 'subtype_of_property', 'number_of_rooms',
       'living_area', 'equipped_kitchen', 'furnished', 'open_fire', 'terrace',
       'garden', 'number_of_facades', 'swimming_pool', 'state_of_building',
       'garden_surface', 'terrace_surface', 'price_per_m2'],
      dtype='object')

Handling missing values

In [151]:
display(df_clean[df_clean['property_id'].isnull()])
print(f"Price is null in {df_clean['price'].isnull().sum()} rows")
df_clean = df_clean.dropna(subset=['price'])
display(display(df_clean[df_clean.isnull().all(axis=1)]))

,property_id,locality_name,postal_code,price,type_of_property,subtype_of_property,number_of_rooms,living_area,equipped_kitchen,furnished,open_fire,terrace,garden,number_of_facades,swimming_pool,state_of_building,garden_surface,terrace_surface,price_per_m2


Price is null in 0 rows


,property_id,locality_name,postal_code,price,type_of_property,subtype_of_property,number_of_rooms,living_area,equipped_kitchen,furnished,open_fire,terrace,garden,number_of_facades,swimming_pool,state_of_building,garden_surface,terrace_surface,price_per_m2


None

In [152]:
# drop rows where all elements are missing
df_clean.dropna(how='all', inplace=True)

display(df_clean.head())

,property_id,locality_name,postal_code,price,type_of_property,subtype_of_property,number_of_rooms,living_area,equipped_kitchen,furnished,open_fire,terrace,garden,number_of_facades,swimming_pool,state_of_building,garden_surface,terrace_surface,price_per_m2
0,VWD15538,Seneffe,7180,410000,House,Villa,4.0,175.0,1,0,0,1,1,4.0,0,Normal,NaN,NaN,2342.857143
1,RBU63284,Hennuyères,7090,750000,House,House,4.0,550.0,0,0,0,1,1,4.0,0,Normal,1085.0,30.0,1363.636364
2,VBD47252,La Louvière,7100,175000,Appartment,Flat,2.0,76.0,1,0,0,1,0,2.0,0,Unknown,NaN,NaN,2302.631579
3,RBU62400,Solre-sur-Sambre,6560,80000,House,House,3.0,235.0,0,0,0,1,0,3.0,0,To be renovated,NaN,30.0,340.425532
4,RBU62399,Mons,7000,195000,House,House,2.0,103.0,0,0,0,1,1,NaN,0,Unknown,80.0,10.0,1893.203883


Removing duplicates

In [153]:
print(f"Number of duplicate rows: {df_clean.duplicated().sum()}")
df_clean = df_clean.drop_duplicates()

Number of duplicate rows: 0


In [154]:
print(df_clean.duplicated(subset=['property_id']).sum())

0


Handling outliers

But in many real datasets (e.g., real-estate, prices, salaries, transactions), outliers are real, not mistakes.
If you remove too many, you hurt your model.

So for real-estate data (houses can be expensive). I opt for Winsorization ( Cap outliers instead of removing them)

Instead of deleting, you clip extreme values:

In [155]:
display(df_clean.head())
print(df_clean.columns)
print(df_clean.dtypes)
df_clean['price'] = df_clean['price'].clip(
    lower=df_clean['price'].quantile(0.01),
    upper=df_clean['price'].quantile(0.99)
)

df_clean['garden_surface'] = df_clean['garden_surface'].clip(
    lower=df_clean['garden_surface'].quantile(0.01),
    upper=df_clean['garden_surface'].quantile(0.99)
)

df_clean['terrace_surface'] = df_clean['terrace_surface'].clip(
    lower=df_clean['terrace_surface'].quantile(0.01),
    upper=df_clean['terrace_surface'].quantile(0.99)
)

df_clean['living_area'] = df_clean['living_area'].clip(
    lower=df_clean['living_area'].quantile(0.01),
    upper=df_clean['living_area'].quantile(0.99)
)


display(df_clean.head())



,property_id,locality_name,postal_code,price,type_of_property,subtype_of_property,number_of_rooms,living_area,equipped_kitchen,furnished,open_fire,terrace,garden,number_of_facades,swimming_pool,state_of_building,garden_surface,terrace_surface,price_per_m2
0,VWD15538,Seneffe,7180,410000,House,Villa,4.0,175.0,1,0,0,1,1,4.0,0,Normal,NaN,NaN,2342.857143
1,RBU63284,Hennuyères,7090,750000,House,House,4.0,550.0,0,0,0,1,1,4.0,0,Normal,1085.0,30.0,1363.636364
2,VBD47252,La Louvière,7100,175000,Appartment,Flat,2.0,76.0,1,0,0,1,0,2.0,0,Unknown,NaN,NaN,2302.631579
3,RBU62400,Solre-sur-Sambre,6560,80000,House,House,3.0,235.0,0,0,0,1,0,3.0,0,To be renovated,NaN,30.0,340.425532
4,RBU62399,Mons,7000,195000,House,House,2.0,103.0,0,0,0,1,1,NaN,0,Unknown,80.0,10.0,1893.203883


Index(['property_id', 'locality_name', 'postal_code', 'price',
       'type_of_property', 'subtype_of_property', 'number_of_rooms',
       'living_area', 'equipped_kitchen', 'furnished', 'open_fire', 'terrace',
       'garden', 'number_of_facades', 'swimming_pool', 'state_of_building',
       'garden_surface', 'terrace_surface', 'price_per_m2'],
      dtype='object')
property_id             object
locality_name           object
postal_code              int64
price                    int64
type_of_property        object
subtype_of_property     object
number_of_rooms        float64
living_area            float64
equipped_kitchen         int64
furnished                int64
open_fire                int64
terrace                  int64
garden                   int64
number_of_facades      float64
swimming_pool            int64
state_of_building       object
garden_surface         float64
terrace_surface        float64
price_per_m2           float64
dtype: object


/var/folders/hd/3b5ptsrd3jn3s9zm5flfrg3h0000gn/T/ipykernel_7115/2735847302.py:4: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean['price'] = df_clean['price'].clip(


,property_id,locality_name,postal_code,price,type_of_property,subtype_of_property,number_of_rooms,living_area,equipped_kitchen,furnished,open_fire,terrace,garden,number_of_facades,swimming_pool,state_of_building,garden_surface,terrace_surface,price_per_m2
0,VWD15538,Seneffe,7180,410000,House,Villa,4.0,175.0,1,0,0,1,1,4.0,0,Normal,NaN,NaN,2342.857143
1,RBU63284,Hennuyères,7090,750000,House,House,4.0,550.0,0,0,0,1,1,4.0,0,Normal,1085.0,30.0,1363.636364
2,VBD47252,La Louvière,7100,175000,Appartment,Flat,2.0,76.0,1,0,0,1,0,2.0,0,Unknown,NaN,NaN,2302.631579
3,RBU62400,Solre-sur-Sambre,6560,80000,House,House,3.0,235.0,0,0,0,1,0,3.0,0,To be renovated,NaN,30.0,340.425532
4,RBU62399,Mons,7000,195000,House,House,2.0,103.0,0,0,0,1,1,NaN,0,Unknown,80.0,10.0,1893.203883


Fixing data types

In [156]:
# Drop Price per m2, this was derived from target and can cause data leakage
if 'price_per_m2' in df_clean.columns:
    df_clean = df_clean.drop(columns=['price_per_m2'])
display(df_clean.columns)
display(df_clean.dtypes)
display(df_clean.head())

Index(['property_id', 'locality_name', 'postal_code', 'price',
       'type_of_property', 'subtype_of_property', 'number_of_rooms',
       'living_area', 'equipped_kitchen', 'furnished', 'open_fire', 'terrace',
       'garden', 'number_of_facades', 'swimming_pool', 'state_of_building',
       'garden_surface', 'terrace_surface'],
      dtype='object')

property_id             object
locality_name           object
postal_code              int64
price                    int64
type_of_property        object
subtype_of_property     object
number_of_rooms        float64
living_area            float64
equipped_kitchen         int64
furnished                int64
open_fire                int64
terrace                  int64
garden                   int64
number_of_facades      float64
swimming_pool            int64
state_of_building       object
garden_surface         float64
terrace_surface        float64
dtype: object

,property_id,locality_name,postal_code,price,type_of_property,subtype_of_property,number_of_rooms,living_area,equipped_kitchen,furnished,open_fire,terrace,garden,number_of_facades,swimming_pool,state_of_building,garden_surface,terrace_surface
0,VWD15538,Seneffe,7180,410000,House,Villa,4.0,175.0,1,0,0,1,1,4.0,0,Normal,NaN,NaN
1,RBU63284,Hennuyères,7090,750000,House,House,4.0,550.0,0,0,0,1,1,4.0,0,Normal,1085.0,30.0
2,VBD47252,La Louvière,7100,175000,Appartment,Flat,2.0,76.0,1,0,0,1,0,2.0,0,Unknown,NaN,NaN
3,RBU62400,Solre-sur-Sambre,6560,80000,House,House,3.0,235.0,0,0,0,1,0,3.0,0,To be renovated,NaN,30.0
4,RBU62399,Mons,7000,195000,House,House,2.0,103.0,0,0,0,1,1,NaN,0,Unknown,80.0,10.0


In [157]:
missing_count = df_clean.isna().sum()
missing_percent = (df_clean.isna().mean() * 100).round(2)

# Combine into one table
missing_table = pd.DataFrame({
    "missing_count": missing_count,
    "missing_percent": missing_percent
})

# Sort the result from most missing to least missing
missing_table = missing_table.sort_values("missing_percent", ascending=False)

print(missing_table)

                     missing_count  missing_percent
garden_surface                5237            76.45
terrace_surface               4257            62.15
number_of_facades             1850            27.01
living_area                    398             5.81
number_of_rooms                124             1.81
open_fire                        0             0.00
state_of_building                0             0.00
swimming_pool                    0             0.00
garden                           0             0.00
terrace                          0             0.00
property_id                      0             0.00
locality_name                    0             0.00
equipped_kitchen                 0             0.00
subtype_of_property              0             0.00
type_of_property                 0             0.00
price                            0             0.00
postal_code                      0             0.00
furnished                        0             0.00


| Column            | Missing | What to do                        | Why                        |
| ----------------- | ------- | --------------------------------- | -------------------------- |
| garden_surface    | 76%     | 0 if garden=0, median if garden=1 | logical missing            |
| terrace_surface   | 62%     | same logic                        | logical missing            |
| number_of_facades | 27%     | impute median by property type    | important structural value |
| price             | 5.8%    | drop rows                         | target variable            |
| living_area       | 5.8%    | median by subtype                 | strong predictor           |
| number_of_rooms   | 1.8%    | median                            | weakly missing             |


In [158]:
df_clean.head()

,property_id,locality_name,postal_code,price,type_of_property,subtype_of_property,number_of_rooms,living_area,equipped_kitchen,furnished,open_fire,terrace,garden,number_of_facades,swimming_pool,state_of_building,garden_surface,terrace_surface
0,VWD15538,Seneffe,7180,410000,House,Villa,4.0,175.0,1,0,0,1,1,4.0,0,Normal,NaN,NaN
1,RBU63284,Hennuyères,7090,750000,House,House,4.0,550.0,0,0,0,1,1,4.0,0,Normal,1085.0,30.0
2,VBD47252,La Louvière,7100,175000,Appartment,Flat,2.0,76.0,1,0,0,1,0,2.0,0,Unknown,NaN,NaN
3,RBU62400,Solre-sur-Sambre,6560,80000,House,House,3.0,235.0,0,0,0,1,0,3.0,0,To be renovated,NaN,30.0
4,RBU62399,Mons,7000,195000,House,House,2.0,103.0,0,0,0,1,1,NaN,0,Unknown,80.0,10.0


In [159]:
if 'property_id' in df_clean.columns:
    df_clean = df_clean.drop(columns=['property_id'])


In [160]:
object_cols = df_clean.select_dtypes(include="object").columns
for col in object_cols:
    df_clean[col] = df_clean[col].str.lower().str.strip()
df_clean.info()
df_clean.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6850 entries, 0 to 6849
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   locality_name        6850 non-null   object 
 1   postal_code          6850 non-null   int64  
 2   price                6850 non-null   int64  
 3   type_of_property     6850 non-null   object 
 4   subtype_of_property  6850 non-null   object 
 5   number_of_rooms      6726 non-null   float64
 6   living_area          6452 non-null   float64
 7   equipped_kitchen     6850 non-null   int64  
 8   furnished            6850 non-null   int64  
 9   open_fire            6850 non-null   int64  
 10  terrace              6850 non-null   int64  
 11  garden               6850 non-null   int64  
 12  number_of_facades    5000 non-null   float64
 13  swimming_pool        6850 non-null   int64  
 14  state_of_building    6850 non-null   object 
 15  garden_surface       1613 non-null   f

,locality_name,postal_code,price,type_of_property,subtype_of_property,number_of_rooms,living_area,equipped_kitchen,furnished,open_fire,terrace,garden,number_of_facades,swimming_pool,state_of_building,garden_surface,terrace_surface
0,seneffe,7180,410000,house,villa,4.0,175.0,1,0,0,1,1,4.0,0,normal,NaN,NaN
1,hennuyères,7090,750000,house,house,4.0,550.0,0,0,0,1,1,4.0,0,normal,1085.0,30.0
2,la louvière,7100,175000,appartment,flat,2.0,76.0,1,0,0,1,0,2.0,0,unknown,NaN,NaN
3,solre-sur-sambre,6560,80000,house,house,3.0,235.0,0,0,0,1,0,3.0,0,to be renovated,NaN,30.0
4,mons,7000,195000,house,house,2.0,103.0,0,0,0,1,1,NaN,0,unknown,80.0,10.0


In [161]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6850 entries, 0 to 6849
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   locality_name        6850 non-null   object 
 1   postal_code          6850 non-null   int64  
 2   price                6850 non-null   int64  
 3   type_of_property     6850 non-null   object 
 4   subtype_of_property  6850 non-null   object 
 5   number_of_rooms      6726 non-null   float64
 6   living_area          6452 non-null   float64
 7   equipped_kitchen     6850 non-null   int64  
 8   furnished            6850 non-null   int64  
 9   open_fire            6850 non-null   int64  
 10  terrace              6850 non-null   int64  
 11  garden               6850 non-null   int64  
 12  number_of_facades    5000 non-null   float64
 13  swimming_pool        6850 non-null   int64  
 14  state_of_building    6850 non-null   object 
 15  garden_surface       1613 non-null   f

In [162]:
def is_missing(df):

    missing_count = df.isna().sum()
    missing_percent = (df.isna().mean() * 100).round(2)

    # Combine into one table
    missing_table = pd.DataFrame({
        "missing_count": missing_count,
        "missing_percent": missing_percent
    })

    # Sort the result from most missing to least missing
    missing_table = missing_table.sort_values("missing_percent", ascending=False)

    print(missing_table)

is_missing(df_clean)

                     missing_count  missing_percent
garden_surface                5237            76.45
terrace_surface               4257            62.15
number_of_facades             1850            27.01
living_area                    398             5.81
number_of_rooms                124             1.81
terrace                          0             0.00
state_of_building                0             0.00
swimming_pool                    0             0.00
garden                           0             0.00
locality_name                    0             0.00
open_fire                        0             0.00
postal_code                      0             0.00
equipped_kitchen                 0             0.00
subtype_of_property              0             0.00
type_of_property                 0             0.00
price                            0             0.00
furnished                        0             0.00


In [163]:
if "garden" in df_clean.columns and "garden_surface" in df_clean.columns:
    # garden = 0 → garden_surface = 0
    df_clean.loc[(df_clean["garden"] == 0) & (df_clean["garden_surface"].isna()), "garden_surface"] = 0

    # garden = 1 → median
    median_garden = df_clean.loc[df_clean["garden"] == 1, "garden_surface"].median()
    df_clean.loc[(df_clean["garden"] == 1) & (df_clean["garden_surface"].isna()), "garden_surface"] = median_garden

if "terrace" in df_clean.columns and "terrace_surface" in df_clean.columns:
    df_clean.loc[(df_clean["terrace"] == 0) & (df_clean["terrace_surface"].isna()), "terrace_surface"] = 0

    median_terrace = df_clean.loc[df_clean["terrace"] == 1, "terrace_surface"].median()
    df_clean.loc[(df_clean["terrace"] == 1) & (df_clean["terrace_surface"].isna()), "terrace_surface"] = median_terrace

is_missing(df_clean)

                     missing_count  missing_percent
number_of_facades             1850            27.01
living_area                    398             5.81
number_of_rooms                124             1.81
locality_name                    0             0.00
terrace                          0             0.00
garden_surface                   0             0.00
state_of_building                0             0.00
swimming_pool                    0             0.00
garden                           0             0.00
furnished                        0             0.00
open_fire                        0             0.00
postal_code                      0             0.00
equipped_kitchen                 0             0.00
subtype_of_property              0             0.00
type_of_property                 0             0.00
price                            0             0.00
terrace_surface                  0             0.00


number_of_facades (27% missing)
Best approach: impute using grouped median by property type.
Why?

Houses usually have 3–4 façades

Apartments usually have 2

In [164]:
if "number_of_facades" in df_clean.columns:
    df_clean["number_of_facades"] = df_clean.groupby("type_of_property")["number_of_facades"].transform(
        lambda x: x.fillna(x.median())
    )
    # fallback: fill any remaining with global median
    df_clean["number_of_facades"] = df_clean["number_of_facades"].fillna(
        df_clean["number_of_facades"].median()
    )

is_missing(df_clean)

                     missing_count  missing_percent
living_area                    398             5.81
number_of_rooms                124             1.81
locality_name                    0             0.00
terrace                          0             0.00
garden_surface                   0             0.00
state_of_building                0             0.00
swimming_pool                    0             0.00
number_of_facades                0             0.00
garden                           0             0.00
furnished                        0             0.00
open_fire                        0             0.00
postal_code                      0             0.00
equipped_kitchen                 0             0.00
subtype_of_property              0             0.00
type_of_property                 0             0.00
price                            0             0.00
terrace_surface                  0             0.00


Living_area (5.81% missing)

Best approach: median per property subtype

Why?

Apartments have smaller living_area

Houses vary a lot

Subtype (villa, duplex, penthouse…) is even more precise

In [165]:
if "living_area" in df_clean.columns:
    df_clean["living_area"] = df_clean.groupby("subtype_of_property")["living_area"].transform(
        lambda x: x.fillna(x.median())
    )
    # fallback
    df_clean["living_area"] = df_clean["living_area"].fillna(df_clean["living_area"].median())
is_missing(df_clean)

                     missing_count  missing_percent
number_of_rooms                124             1.81
locality_name                    0             0.00
open_fire                        0             0.00
garden_surface                   0             0.00
state_of_building                0             0.00
swimming_pool                    0             0.00
number_of_facades                0             0.00
garden                           0             0.00
terrace                          0             0.00
furnished                        0             0.00
postal_code                      0             0.00
equipped_kitchen                 0             0.00
living_area                      0             0.00
subtype_of_property              0             0.00
type_of_property                 0             0.00
price                            0             0.00
terrace_surface                  0             0.00


In [166]:
df_clean = df_clean.dropna(subset=["price"])
is_missing(df_clean)

                     missing_count  missing_percent
number_of_rooms                124             1.81
locality_name                    0             0.00
open_fire                        0             0.00
garden_surface                   0             0.00
state_of_building                0             0.00
swimming_pool                    0             0.00
number_of_facades                0             0.00
garden                           0             0.00
terrace                          0             0.00
furnished                        0             0.00
postal_code                      0             0.00
equipped_kitchen                 0             0.00
living_area                      0             0.00
subtype_of_property              0             0.00
type_of_property                 0             0.00
price                            0             0.00
terrace_surface                  0             0.00


number_of_rooms (1.81% missing)

In [167]:
if "number_of_rooms" in df_clean.columns:
    df_clean["number_of_rooms"] = df_clean["number_of_rooms"].fillna(
        df_clean["number_of_rooms"].median()
    )
is_missing(df_clean)

                     missing_count  missing_percent
locality_name                    0              0.0
open_fire                        0              0.0
garden_surface                   0              0.0
state_of_building                0              0.0
swimming_pool                    0              0.0
number_of_facades                0              0.0
garden                           0              0.0
terrace                          0              0.0
furnished                        0              0.0
postal_code                      0              0.0
equipped_kitchen                 0              0.0
living_area                      0              0.0
number_of_rooms                  0              0.0
subtype_of_property              0              0.0
type_of_property                 0              0.0
price                            0              0.0
terrace_surface                  0              0.0


Save to cleaned_data.csv

In [168]:
df_clean.to_csv("../data/2_cleaned/cleaned_data.csv", index=False, encoding="utf-8")

In [170]:
def clean_object_columns(df):
    obj_cols = df.select_dtypes(include="object").columns
    
    for col in obj_cols:
        df[col] = (
            df[col]
            .astype(str)
            .str.lower()
            .str.strip()
            .str.replace(r"\s+", "_", regex=True)   # replace any whitespace
            .replace({"": "unknown", "nan": "unknown"})
            .fillna("unknown")
        )
    
    return df
df_cleaned = clean_object_columns(df_clean)
display(df_cleaned.head(100))

,locality_name,postal_code,price,type_of_property,subtype_of_property,number_of_rooms,living_area,equipped_kitchen,furnished,open_fire,terrace,garden,number_of_facades,swimming_pool,state_of_building,garden_surface,terrace_surface
0,seneffe,7180,410000,house,villa,4.0,175.00,1,0,0,1,1,4.0,0,normal,340.0,17.0
1,hennuyères,7090,750000,house,house,4.0,550.00,0,0,0,1,1,4.0,0,normal,1085.0,30.0
2,la_louvière,7100,175000,appartment,flat,2.0,76.00,1,0,0,1,0,2.0,0,unknown,0.0,17.0
3,solre-sur-sambre,6560,80000,house,house,3.0,235.00,0,0,0,1,0,3.0,0,to_be_renovated,0.0,30.0
4,mons,7000,195000,house,house,2.0,103.00,0,0,0,1,1,3.0,0,unknown,80.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,charleroi,6000,165000,appartment,flat,3.0,120.00,0,0,0,1,0,2.0,0,unknown,0.0,3.0
96,tourpes,7904,350000,house,house,4.0,300.00,0,0,0,1,1,4.0,0,excellent,340.0,17.0
97,mouscron,7700,649000,house,villa,4.0,420.00,0,0,0,1,1,4.0,0,excellent,800.0,17.0
98,péronnes,7134,249000,house,house,5.0,210.00,0,0,0,1,1,2.0,0,new,250.0,25.0
